# Serie 3
## Esercizio 1

In [1]:
class Worker implements Runnable {
        public static boolean isRunning = false;
        public static int finished = 0;

        private int count = 0;
        private final int id;
        private final Random random;

        public Worker(final int id) {
                this.id = id;
                this.random = new Random();
        }

        @Override
        public void run() {
                System.out.println("Worker" + id + " waiting to start");
                while (!isRunning) {
                        // Wait!
                }

                System.out.println("Worker" + id + " started");
                for (int i = 0; i < 10; i++) {
                        count += random.nextInt(40) + 10;
                        try {
                                Thread.sleep(random.nextInt(151) + 100);
                        } catch (final InterruptedException e) {
                                e.printStackTrace();
                        }
                }

                System.out.println("Worker" + id + " finished");
                finished++;
        }

        public void printResult() {
                System.out.println("Worker" + id + " reached " + count);
        }
}


final List <Worker> allWorkers = new ArrayList < > ();
final List <Thread> allThread = new ArrayList < > ();
for (int i = 1; i <= 10; i++) {
    final Worker target = new Worker(i);
    allWorkers.add(target);
    final Thread e = new Thread(target);
    allThread.add(e);
    e.start();
}

try {
    Thread.sleep(1000);
} catch (final InterruptedException e){

}

System.out.println("Main thread starting the race!");
Worker.isRunning = true;

boolean timedOut = false;

Thread t = new Thread(() -> {
    try {
        Thread.sleep(10000);
        System.out.println("Timed out.");
        timedOut = true;
        allThread.forEach(Thread::interrupt);
    } catch (InterruptedException e) {
    }
});

t.start();

while (Worker.finished < allWorkers.size() && !timedOut){
    // Wait
    Thread.sleep(50);
}

if(!timedOut){
    for (Worker worker: allWorkers){
        worker.printResult();
    }

    for (final Thread thread: allThread){
        try {
            thread.join();
        } catch (final InterruptedException e) {
            e.printStackTrace();
        }
    }
}

Worker7 waiting to start
Worker10 waiting to start
Worker5 waiting to start
Worker1 waiting to start
Worker2 waiting to start
Worker8 waiting to start
Worker6 waiting to start
Worker3 waiting to start
Worker4 waiting to start
Worker9 waiting to start
Main thread starting the race!
Timed out.


### Problematica
Il thread principale va in uno stato di hang perché Worker.finished viene messo in una cache diversa, a seconda del thread. Di conseguenza la modifica non è propagata agli altri thread ed i worker vanno in stato di hang.

### Risoluzione

Per ovviare al problema dobbiamo rendere la variabile statica "isRunning" di Worker `volatile`, in modo che questa sia sempre aggiornata in tutti i thread e che quindi non causi
dei cicli while infiniti causati dall'espressione `!isRunning`.

In [2]:
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;
class Worker implements Runnable {
    public volatile static boolean isRunning = false;
    public volatile static int finished = 0;
    public static Lock lock = new ReentrantLock();
    private final int id;
    private final Random random;
    private int count = 0;

    public Worker(final int id) {
        this.id = id;
        this.random = new Random();
    }

    public static int getFinishedWorkers(){
        return finished;
    }

    @Override
    public void run() {
        System.out.println("Worker" + id + " waiting to start");
        while (!isRunning) {
            // Wait!
        }

        System.out.println("Worker" + id + " started");
        for (int i = 0; i < 10; i++) {
            count += random.nextInt(40) + 10;
            try {
                Thread.sleep(random.nextInt(151) + 100);
            } catch (final InterruptedException e) {
                e.printStackTrace();
            }
        }

        System.out.println("Worker" + id + " finished");
        Worker.lock.lock();
        finished++;
        Worker.lock.unlock();
    }

    public void printResult() {
        System.out.println("Worker" + id + " reached " + count);
    }
}

final List <Worker> allWorkers = new ArrayList < > ();
final List <Thread> allThread = new ArrayList < > ();
for (int i = 1; i <= 10; i++) {
    final Worker target = new Worker(i);
    allWorkers.add(target);
    final Thread e = new Thread(target);
    allThread.add(e);
    e.start();
}

try {
    Thread.sleep(1000);
} catch (final InterruptedException e){

}

System.out.println("Main thread starting the race!");
Worker.isRunning = true;

Thread t = new Thread(() -> {
    try {
        Thread.sleep(10000);
        System.out.println("Timed out.");
        System.exit(1);
    } catch (InterruptedException e) {
    }
});

t.start();


while (Worker.getFinishedWorkers() < allWorkers.size()){
    // Wait
}

t.interrupt();

for (Worker worker: allWorkers){
    worker.printResult();
}

for (final Thread thread: allThread){
    try {
        thread.join();
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }
}

Worker1 waiting to start
Worker2 waiting to start
Worker3 waiting to start
Worker4 waiting to start
Worker5 waiting to start
Worker6 waiting to start
Worker7 waiting to start
Worker9 waiting to start
Worker8 waiting to start
Worker10 waiting to start
Main thread starting the race!
Worker4 started
Worker9 started
Worker10 started
Worker1 started
Worker7 started
Worker6 started
Worker2 started
Worker5 started
Worker8 started
Worker3 started
Worker9 finished
Worker5 finished
Worker7 finished
Worker1 finished
Worker8 finished
Worker4 finished
Worker10 finished
Worker6 finished
Worker2 finished
Worker3 finished
Worker1 reached 263
Worker2 reached 286
Worker3 reached 231
Worker4 reached 304
Worker5 reached 288
Worker6 reached 339
Worker7 reached 305
Worker8 reached 339
Worker9 reached 208
Worker10 reached 360


Vediamo quindi che, una volta corretto il codice inserendo `volatile`, il tutto funziona nel modo corretto.

## Esercizio 2


### Prima versione
La presente versione non utilizza nessuna opzione di sincronizzazione, di conseguenza i thread vanno in uno stato di hang ed il thread principale non termina mai.

### Seconda versione
Utilizzando la keyword `volatile` su `Counter.value` otteniamo un risultato corretto

In [3]:
class Counter {
    public static volatile int value = 0;
}

class Sensor extends Thread {
    private int threshold;

    public Sensor(int threshold){
        this.threshold = threshold;
    }
    @Override
    public void run() {
        while(Counter.value < threshold){ }
        System.out.println(String.format("Counter.value = %d  > %d - resetting!", Counter.value, threshold));
        Counter.value = 0;
    }
}

ArrayList<Sensor> sensors = new ArrayList<>();
for(int i = 0; i<10; i++) {
    sensors.add(new Sensor((i+1) * 10));
}

for(Sensor sensor : sensors){
    System.out.println("Starting");
    sensor.start();
}

Random r = new Random();
while(Counter.value < 120){
    //Counter.lock.lock();
    Counter.value += r.nextInt(7) + 1;
    //Counter.lock.unlock();
    try {
        Thread.sleep(r.nextInt(5) + 5);
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
}

for(Sensor sensor : sensors){
    sensor.join();
}

System.out.println("Programma terminato");

Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Counter.value = 14  > 10 - resetting!
Counter.value = 21  > 20 - resetting!
Counter.value = 35  > 30 - resetting!
Counter.value = 42  > 40 - resetting!
Counter.value = 51  > 50 - resetting!
Counter.value = 61  > 60 - resetting!
Counter.value = 77  > 70 - resetting!
Counter.value = 84  > 80 - resetting!
Counter.value = 90  > 90 - resetting!
Counter.value = 100  > 100 - resetting!
Programma terminato


Questa versione pare comportarsi in modo corretto. Purtroppo però non esiste nessuna garanzia che l'esecuzione dei thread (e di conseguenza dell'output) sia sequenziale. Potrebbe accadere di ottenere un risultato simile a questo:
```
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Counter.value = 12  > 10 - resetting!
Counter.value = 23  > 20 - resetting!
Counter.value = 45  > 40 - resetting!
Counter.value = 36  > 30 - resetting!
Counter.value = 52  > 50 - resetting!
Counter.value = 60  > 60 - resetting!
Counter.value = 75  > 70 - resetting!
Counter.value = 80  > 80 - resetting!
Counter.value = 93  > 90 - resetting!
Counter.value = 103  > 100 - resetting!
Programma terminato
```


### Terza versione (Utilizzo di variabili atomiche)
Per ovviare al problema possiamo utilizzare delle variabili atomiche, in questo modo l'incremento del contatore sarà sempre un operazione atomica.

In [4]:
import java.util.concurrent.atomic.AtomicInteger;

class Counter {
    public static volatile AtomicInteger value = new AtomicInteger(0);
}

class Sensor extends Thread {
    private int threshold;

    public Sensor(int threshold){
        this.threshold = threshold;
    }
    @Override
    public void run() {
        while (Counter.value.get() < threshold) { }
        System.out.println(String.format("Counter.value = %d  > %d - resetting!", Counter.value.get(), threshold));
        Counter.value.set(0);
    }
}

ArrayList<Sensor> sensors = new ArrayList<>();
for(int i = 0; i<10; i++) {
    sensors.add(new Sensor((i+1) * 10));
}

for(Sensor sensor : sensors){
    System.out.println("Starting");
    sensor.start();
}

Random r = new Random();
while(Counter.value.get() < 120){
    Counter.value.getAndAdd(r.nextInt(7) + 1);
    try {
        Thread.sleep(r.nextInt(5) + 5);
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
}
for(Sensor sensor: sensors){
    sensor.join();
}
System.out.println("Programma terminato");

Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Counter.value = 15  > 10 - resetting!
Counter.value = 22  > 20 - resetting!
Counter.value = 30  > 30 - resetting!
Counter.value = 42  > 40 - resetting!
Counter.value = 51  > 50 - resetting!
Counter.value = 61  > 60 - resetting!
Counter.value = 70  > 70 - resetting!
Counter.value = 85  > 80 - resetting!
Counter.value = 90  > 90 - resetting!
Counter.value = 102  > 100 - resetting!
Programma terminato


Il risultato è quello aspettato, e le operazioni hanno garanzia di atomicità. Il programma è quindi *thread safe*.

### Quarta versione (Utilizzo di Lock Espliciti)

In [5]:
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;

class Counter {
    public static volatile int value = 0;
    public static Lock lock = new ReentrantLock();
}

class Sensor extends Thread {
    private int threshold;

    public Sensor(int threshold){
        this.threshold = threshold;
    }
    @Override
    public void run() {
        while(Counter.value < threshold){ }
        System.out.println(String.format("Counter.value = %d  > %d - resetting!", Counter.value, threshold));
        Counter.lock.lock();
        Counter.value = 0;
        Counter.lock.unlock();
    }
}

ArrayList<Sensor> sensors = new ArrayList<>();
for(int i = 0; i<10; i++) {
    sensors.add(new Sensor((i+1) * 10));
}

for(Sensor sensor : sensors){
    System.out.println("Starting");
    sensor.start();
}

Random r = new Random();
while(Counter.value < 120){
    Counter.lock.lock();
    Counter.value += r.nextInt(7) + 1;
    Counter.lock.unlock();
    try {
        Thread.sleep(r.nextInt(5) + 5);
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
}
for(Sensor sensor : sensors){
    sensor.join();
}
System.out.println("Programma terminato");

Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Counter.value = 15  > 10 - resetting!
Counter.value = 22  > 20 - resetting!
Counter.value = 31  > 30 - resetting!
Counter.value = 46  > 40 - resetting!
Counter.value = 51  > 50 - resetting!
Counter.value = 66  > 60 - resetting!
Counter.value = 70  > 70 - resetting!
Counter.value = 88  > 80 - resetting!
Counter.value = 92  > 90 - resetting!
Counter.value = 101  > 100 - resetting!
Programma terminato


Sfruttando i lock esplicit possiamo quindi garantire l'atomicità delle operazioni di incremento e reset, rendendo così la nostra applicazione *thread safe*.

### Quinta versione (Utilizzo di ReadWriteLock)
In questa versione utilizzeremo dei ReadWriteLock per rendere il programma più efficiente.

In [6]:
import java.util.concurrent.locks.ReadWriteLock;
import java.util.concurrent.locks.ReentrantReadWriteLock;

class Counter {
    public static volatile int value = 0;
    public static ReadWriteLock lock = new ReentrantReadWriteLock();
}

class Sensor extends Thread {
    private int threshold;

    public Sensor(int threshold){
        this.threshold = threshold;
    }
    @Override
    public void run() {
        while(Counter.value < threshold){ }
        System.out.println(String.format("Counter.value = %d  > %d - resetting!", Counter.value, threshold));
        Counter.lock.writeLock().lock();
        Counter.value = 0;
        Counter.lock.writeLock().unlock();
    }
}

ArrayList<Sensor> sensors = new ArrayList<>();
for(int i = 0; i<10; i++) {
    sensors.add(new Sensor((i+1) * 10));
}

for(Sensor sensor : sensors){
    System.out.println("Starting");
    sensor.start();
}

Random r = new Random();
while(Counter.value < 120){
    Counter.lock.writeLock().lock();
    Counter.value += r.nextInt(7) + 1;
    Counter.lock.writeLock().unlock();

    try {
        Thread.sleep(r.nextInt(5) + 5);
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
}
System.out.println("Programma terminato");

Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
Counter.value = 11  > 10 - resetting!
Counter.value = 23  > 20 - resetting!
Counter.value = 42  > 30 - resetting!
Counter.value = 42  > 40 - resetting!
Counter.value = 52  > 50 - resetting!
Counter.value = 71  > 70 - resetting!
Counter.value = 64  > 60 - resetting!
Counter.value = 84  > 80 - resetting!
Counter.value = 91  > 90 - resetting!
Counter.value = 105  > 100 - resetting!
Programma terminato


In questa ultima versione il programma risulta più efficiente in quanto tutti e 10 i Thread non devono "congelare" la loro esecuzione mentre la variabile `Counter.value` viene cambiata, in quanto questa viene resa `read-only` durante la sua modifica con l'utilizzo di un `lock` sul `WriteLock`.
Solamente le loro operazioni di scrittura sono congelate fino al rilascio del lock.